##### Copyright 2020 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2020 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/bert_experts"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/bert_experts.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/bert_experts.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/bert_experts.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# BERT Experts from TF-Hub

This colab demonstrates how to:
* Load BERT models from [TensorFlow Hub](https://tfhub.dev) that have been trained on different tasks including MNLI, SQuAD, and PubMed
* Use a SentencePiece model using the BERT model vocabulary to tokenize raw text and convert it to ids
* Generate the pooled and sequence output from the token input ids using the loaded model
* Look at the semantic similarity of the pooled outputs of different sentences

#### Note: This colab should be run with a TPU runtime

## Set up and imports

In [ ]:
!pip3 install --quiet tensorflow
!pip3 install --quiet sentencepiece
!pip3 install --quiet tf-models-official

In [ ]:
import numpy as np
import os
import seaborn as sns
from sklearn.metrics import pairwise

import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
import sentencepiece as spm
from official.nlp.bert import tokenization

## TPU Configuration

Let's configure the TPUs to be able to run the BERT model quickly.

Go to **Runtime** → **Change runtime type** to make sure that **TPU** is selected

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
#@title Helper functions for tokenization

def build_input(tokenizer, sentence, max_seq_length):
  """Generate (input_ids, input_mask, segment_ids) for a single sentence."""
  tokens = tokenizer.tokenize(sentence)
  tokens = ["[CLS]"] + tokens + ["[SEP]"]
  ids = tokenizer.convert_tokens_to_ids(tokens)

  # Pad the ids to max sequence length
  pad_len = max_seq_length - len(ids)
  input_ids = ids + [0]*pad_len
  input_mask = [1]*len(ids) + [0]*pad_len

  # Single sentence segment_ids are all 0
  segment_ids = [0]*max_seq_length
  return (input_ids, input_mask, segment_ids)


def build_inputs(tokenizer, sentences, max_seq_length):
  """Generate (input_ids, input_mask, segment_ids) for a batch of sentences."""
  inputs = [build_input(tokenizer, s, max_seq_length) for s in sentences]

  # Slice to batch each input tensor
  input_ids = np.array([x[0] for x in inputs], dtype=np.int32)
  input_masks = np.array([x[1] for x in inputs], dtype=np.int32)
  segment_ids = np.array([x[2] for x in inputs], dtype=np.int32)
  return [input_ids, input_masks, segment_ids]


def reconstruct_tokens(tokenizer, ids):
  """Map the input_ids from a batch of bert ids to tokens."""
  batched_input_ids = ids[0]
  tokens = [tokenizer.convert_ids_to_tokens(input_ids) for input_ids in batched_input_ids]
  return tokens


def plot_similarity(features, labels):
  """Plot a similarity matrix of the embeddings."""
  cos_sim = pairwise.cosine_similarity(features)
  sns.set(font_scale=1.2)
  cbar_kws=dict(use_gridspec=False, location="left")
  g = sns.heatmap(
      cos_sim, xticklabels=labels, yticklabels=labels,
      vmin=0, vmax=1, cmap="Blues", cbar_kws=cbar_kws)
  g.tick_params(labelright=True, labelleft=False)
  g.set_yticklabels(labels, rotation=0)
  g.set_title("Semantic Textual Similarity")

In [ ]:
#@title Configure the model { run: "auto" }
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/1" # @param {type: "string"} ["https://tfhub.dev/google/experts/bert/wiki_books/1", "https://tfhub.dev/google/experts/bert/wiki_books/mnli/1", "https://tfhub.dev/google/experts/bert/wiki_books/qnli/1", "https://tfhub.dev/google/experts/bert/wiki_books/qqp/1", "https://tfhub.dev/google/experts/bert/wiki_books/squad2/1", "https://tfhub.dev/google/experts/bert/wiki_books/sst2/1",  "https://tfhub.dev/google/experts/bert/pubmed/1", "https://tfhub.dev/google/experts/bert/pubmed/squad2/1"]
MAX_SEQUENCE_LENGTH =  512 #@param {type: "integer"}

## Sentences

Let's take some sentences from Wikipedia to run through model

In [ ]:
sentences = [
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Among the singles released from the album were the songs \"Be My Lover\" and \"Hard To Stay Awake\".",
  "Riccardo Zegna is an Italian jazz musician.",
  "Rajko Maksimović is a composer, writer, and music pedagogue.",
  "One of the most significant Serbian composers of our time, Maksimović has been and remains active in creating works for different ensembles.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]

## Run the model

We'll load the model from TF-Hub, tokenize our sentences using a SentencePiece tokenizer built from our model vocab, then feed in the tokenized sentences to the model. To keep this colab fast and simple, we'll use a single TPU core to compute the BERT embeddings. To learn how to use all the TPU devices available, see the TensorFlow [TPU](https://www.tensorflow.org/guide/tpu) and [Distributed Training](https://www.tensorflow.org/guide/distributed_training) guides.

In [ ]:
with tf.device("/device:TPU:0"):
  bert = hub.load(BERT_MODEL)
  vocab_path = bert.vocab_file.asset_path.numpy()
  tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=bert.do_lower_case)
  inputs = build_inputs(tokenizer, sentences, MAX_SEQUENCE_LENGTH)
  pooled_output, sequence_output = bert(inputs)

In [ ]:
print("Sentences:")
print(sentences)

print("\nTokenized sentences:")
print([tokenizer.tokenize(s) for s in sentences])

print("\nBERT inputs:")
print(inputs)

print("\nPooled embeddings:")
print(pooled_output)

print("\nPer token embeddings:")
print(sequence_output)

## Semantic similarity

Now let's take a look at the `pooled_output` embeddings of our sentences and compare how similar they are across sentences.

In [ ]:
plot_similarity(pooled_output, sentences)

## Learn more

* Find more BERT models on [TensorFlow Hub](https://tfhub.dev)
* This notebook demonstrates simple inference with BERT, you can find a more advanced tutorial about fine-tuning BERT at [tensorflow.org/official_models/fine_tuning_bert](https://www.tensorflow.org/official_models/fine_tuning_bert)
* We used just one TPU chip to run the model, you can learn more about how to load models using tf.distribute at [tensorflow.org/tutorials/distribute/save_and_load](https://www.tensorflow.org/tutorials/distribute/save_and_load)